In [2]:
!pip install -q torch==2.1

In [1]:
!pip install numpy==1.24.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 25.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2


In [1]:
!pip install -q TTS==0.22.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 65.2 MB/s eta 0:00:00
  Preparing metadata (setup.py

In [4]:
!pip install pydub

In [6]:

import torch
import os
from scipy.io.wavfile import write
from pydub import AudioSegment

def text_to_speech(paragraph, count=0):
    # Load pre-trained Tacotron2 model
    tacotron2 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tacotron2', model_math='fp16')
    tacotron2 = tacotron2.to('cuda')
    tacotron2.eval()

    # Load pre-trained WaveGlow model
    waveglow = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow', model_math='fp16')
    waveglow = waveglow.remove_weightnorm(waveglow)
    waveglow = waveglow.to('cuda')
    waveglow.eval()

    # Prepare the input text
    utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')

    # Split the text into sentences
    sentences = paragraph.split('. ')

    # Initialize an empty audio segment
    combined = AudioSegment.empty()

    # Generate the speech for each sentence
    for i, sentence in enumerate(sentences):
        sequences, lengths = utils.prepare_input_sequence([sentence])

        with torch.no_grad():
            mel, _, _ = tacotron2.infer(sequences, lengths)
            audio = waveglow.infer(mel)
            audio_numpy = audio[0].data.cpu().numpy()
            rate = 22050

        # Save the audio for each sentence
        filename = f"audio_{i}.wav"
        write(filename, rate, audio_numpy)

        # Load the audio file
        audio = AudioSegment.from_wav(filename)

        # Append the audio file to the combined audio
        combined += audio

        # Add a pause (500 ms of silence)
        pause = AudioSegment.silent(duration=500)
        combined += pause

        # Delete the individual audio file
        os.remove(filename)

    # Export the combined audio file
    combined.export(f"/content/{count}-combined_audio.wav", format='wav')
text_to_speech("Terminal illness or end-stage disease is a disease that cannot be cured or adequately treated and is expected to result in the death of the patient. This term is more commonly used for progressive diseases such as cancer, dementia or advanced heart disease than for injury. In popular use, it indicates a disease that will progress until death with near absolute certainty, regardless of treatment.", count=0)


Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [3]:
!python --version

Python 3.10.12
